In [90]:
import os
import json
import pandas as pd
import sys

def load_json_files(directory):
    data_list = []

    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.endswith(".json"):
                file_path = os.path.join(root, filename)
                try:
                    with open(file_path, "r") as f:
                        data = json.load(f)
                        data_list.append(data)
                except Exception as e:
                    print(f"Error reading {file_path}: {e}", file=sys.stderr)

    if not data_list:
        print("No valid JSON files found.", file=sys.stderr)
        return None

    df = pd.DataFrame(data_list)
    sorted_df = df.sort_values(by=["m", "k", "n","world_size"])
    return sorted_df


In [91]:
directory = "../slurm_logs"
df = load_json_files(directory)

In [92]:
df

,m,n,k,algorithm,world_size,communication_sms,streamk_sms,success,triton_tflops
0,4864,8256,4096,all_scatter,1,40,64,True,17.390499
1,4864,8256,4096,all_reduce,1,40,64,True,12.067431
3,4864,8256,4096,all_reduce,2,40,64,True,4.563553
6,4864,8256,4096,all_scatter,2,40,64,True,14.928498
2,4864,8256,4096,all_reduce,4,40,64,True,0.571679
7,4864,8256,4096,all_scatter,4,40,64,True,5.121747
4,4864,8256,4096,all_scatter,8,40,64,True,3.615865
5,4864,8256,4096,all_reduce,8,40,64,True,0.262839
